In [1]:
import numpy as np
import pandas as pd

In [2]:
from pathlib import Path
root=Path("data")
root.mkdir(exist_ok=True)
path=Path(root)/"IMDB Dataset.csv"

In [3]:
temp_df = pd.read_csv(path)

In [4]:
df = temp_df.iloc[:10000]

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df.drop_duplicates(inplace=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10992\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [7]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [8]:
df['review'] = df['review'].apply(remove_tags)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10992\2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [9]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10992\740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [11]:
import nltk
from nltk.corpus import stopwords
nltk.data.path.append(str(root))
sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10992\628050437.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [12]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [13]:
import gensim

In [14]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [15]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))
    

In [16]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [17]:
model.build_vocab(story)

In [18]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


(5849287, 6186875)

In [19]:
len(model.wv.index_to_key)

31845

In [20]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [21]:
document_vector(df['review'].values[0])

array([-0.03462921,  0.38774997,  0.18104526, -0.08596788, -0.11056951,
       -0.4909235 ,  0.1509332 ,  0.86722684, -0.25628507, -0.01388916,
       -0.05740634, -0.48965135,  0.04904943,  0.16325669,  0.03510778,
       -0.20886903,  0.09118412, -0.5500364 ,  0.07189064, -0.67463267,
        0.13287872,  0.03947803,  0.19930206, -0.2683169 , -0.17954284,
       -0.1183091 , -0.28658605, -0.28820693, -0.36004186,  0.01654373,
        0.41675395, -0.0346861 ,  0.01388949, -0.2766605 , -0.20110959,
        0.23767677,  0.11696669, -0.38099322, -0.2349536 , -0.73094827,
        0.13175398, -0.27853978, -0.2886888 ,  0.06996626,  0.4326764 ,
       -0.17013861, -0.4093762 , -0.09049979,  0.13607569,  0.2528328 ,
        0.08461122, -0.2803596 , -0.32056874, -0.22708553, -0.41027793,
        0.07921705,  0.34352836,  0.08315121, -0.2835145 ,  0.07253779,
        0.03578969,  0.177326  , -0.10316686,  0.03245676, -0.4259877 ,
        0.40222818,  0.00884421,  0.21078031, -0.5913851 ,  0.24

In [22]:
from tqdm import tqdm

In [23]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [03:41<00:00, 45.10it/s]


In [24]:
X = np.array(X)

In [25]:
X[0]

array([-0.03462921,  0.38774997,  0.18104526, -0.08596788, -0.11056951,
       -0.4909235 ,  0.1509332 ,  0.86722684, -0.25628507, -0.01388916,
       -0.05740634, -0.48965135,  0.04904943,  0.16325669,  0.03510778,
       -0.20886903,  0.09118412, -0.5500364 ,  0.07189064, -0.67463267,
        0.13287872,  0.03947803,  0.19930206, -0.2683169 , -0.17954284,
       -0.1183091 , -0.28658605, -0.28820693, -0.36004186,  0.01654373,
        0.41675395, -0.0346861 ,  0.01388949, -0.2766605 , -0.20110959,
        0.23767677,  0.11696669, -0.38099322, -0.2349536 , -0.73094827,
        0.13175398, -0.27853978, -0.2886888 ,  0.06996626,  0.4326764 ,
       -0.17013861, -0.4093762 , -0.09049979,  0.13607569,  0.2528328 ,
        0.08461122, -0.2803596 , -0.32056874, -0.22708553, -0.41027793,
        0.07921705,  0.34352836,  0.08315121, -0.2835145 ,  0.07253779,
        0.03578969,  0.177326  , -0.10316686,  0.03245676, -0.4259877 ,
        0.40222818,  0.00884421,  0.21078031, -0.5913851 ,  0.24

In [26]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [27]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [28]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [30]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7786680020030045